In [1]:
import pandas as pd 
import string 
from nltk.corpus import stopwords

/Users/abdulrahman/anaconda3/envs/data/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [4]:
#Get the spam data collection 
df_spam = pd.read_csv('SpamCollection',sep='\t',names=['response','massage'])

In [5]:
df_spam.head()

,response,massage
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
df_spam.describe()


,response,massage
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [7]:
#view response 
df_spam.groupby('response').describe()


massage                                                            \
           count unique                                                top   
response                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
response       
ham        30  
spam        4

In [9]:
#Verify length of the messages and also add it as a new column 
df_spam['length'] = df_spam['massage'].apply(len)

In [10]:
df_spam.head()

,response,massage,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [11]:
#define a function to get rid of stopwords present in the messages
def message_text_process(mess):
    no_punct = [char for char in mess if char not in string.punctuation]
    
    no_punct = ''.join(no_punct)
    
    return [word for word in no_punct.split() if word.lower() not in stopwords.words('english')]


In [12]:
df_spam['massage'].head(5).apply(message_text_process)


0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: massage, dtype: object

In [13]:
#start text processing with vectorizer 
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
#use bag of words by applying the function and fit the data into it
bag_of_word = CountVectorizer(analyzer=message_text_process).fit(df_spam['massage'])

In [15]:
#print length of bag of words stored in the vocabulary_ attribute
print(len(bag_of_word.vocabulary_))

11425


In [16]:

message_bagofword = bag_of_word.transform(df_spam['massage'])

In [17]:
#apply tfidf transformer and fit the bag of words into it (transformed version)
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer().fit(message_bagofword)

In [19]:
#print shape of the tfidf 
message_tfidf = tfidf.transform(message_bagofword)
print(message_tfidf.shape)

(5572, 11425)


In [20]:
#choose naive Bayes model to detect the spam and fit the tfidf data into it
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(message_tfidf,df_spam['response'])

In [23]:
#check model for the predicted and expected value say for message#2 and message#5
messge = df_spam['massage'][2]
beg_of_word_for_message = bag_of_word.transform([messge])
end_tfidf = tfidf.transform(beg_of_word_for_message)

print("predicted", spam_detect_model.predict(end_tfidf)[0])
print("expected",df_spam.response[4])

predicted spam
expected ham
